In [ ]:
import requests as req
import pandas as pd

In [ ]:
test = False

In [ ]:
try:
    users = pd.read_csv('../../dados/experimentos/users_gathered.csv', encoding='utf-8')
    df = pd.read_csv('../../dados/experimentos/chess_games_chesscom.csv', encoding='utf-8')
except FileNotFoundError:
    test = True

print(test)

### Seção das funções auxiliares

In [ ]:
def get(s):
    headers = {
    "User-Agent":"duduardo45"
    }
    return req.get(s, headers = headers)

In [ ]:
def extract_black_white(row):

    # separating the variables to be used
    white = row['white']
    black = row['black']
    time_class = row['time_class']
    wplayer = white['username']
    bplayer = black['username']
    wresult = white['result']
    bresult = black['result']
    wrating = white['rating']
    brating = black['rating']
    player_pieces = pd.NA

    # extracting the information relative to the main player
    if wplayer == row['player']:
        if bplayer not in users['username'].unique(): # adds the opponent to users
            users.loc[len(users)] = {
                                    'username':bplayer,
                                    'url' : black['@id'],
                                    'gathered': False,
                                    f'{time_class}_rating': brating,
                                    }
        prating = wrating
        presult = wresult
        player_pieces = 'white'
        opponent = bplayer
        orating = brating
    elif bplayer == row['player']:
        if wplayer not in users['username'].unique():
            users.loc[len(users)] = {
                                    'username':wplayer,
                                    'url' : white['@id'],
                                    'gathered': False,
                                    f'{time_class}_rating': brating,
                                    }
        prating = brating
        presult = bresult
        player_pieces = 'black'
        opponent = wplayer
        orating = wrating
    if wresult == 'win':
        winner = 'white'
        method = bresult
    elif bresult == 'win':
        winner = 'black'
        method = wresult
    elif wresult == bresult:
        winner = 'draw'
        method = presult
    else:
        winner = 'unknown'
        method = wresult + ' + ' + bresult


    # adding the information to the DataFrame
    row['player_rating'] = prating
    row['opponent'] = opponent
    row['opponent_rating'] = orating
    row['player_pieces'] = player_pieces
    row['winner'] = winner
    row['win_method'] = method

    return row

In [ ]:
def adds_user_info(player_games:pd.DataFrame, u):

    most_played_class = player_games['time_class'].mode()[0]
    latest_time = player_games.loc[player_games['end_time'] == player_games['end_time'].max(), 'end_time'].iloc[0]
    users.loc[users['username'] == u, 'most_played_class'] = most_played_class
    users.loc[users['username'] == u, 'latest_game'] = latest_time

    for time_class in player_games['time_class'].unique():
        games = player_games[player_games['time_class'] == time_class]
        amount_played = games.shape[0]
        latest_game = games.loc[games['end_time'] == games['end_time'].max()].iloc[0]
        latest_time = latest_game['end_time']
        p_rating = latest_game['player_rating']
        users.loc[users['username'] == u, f'{time_class}_rating'] = p_rating
        users.loc[users['username'] == u, f'{time_class}_played'] = amount_played
        users.loc[users['username'] == u, f'{time_class}_recent'] = latest_time

In [ ]:
def gather_player(u='Mr-Barros'):

    # requests the api
    archives = "/games/archives"

    user = users.loc[users['username'] == u]

    if user.shape[0] == 0:
        print(f'Usuário "{u}" fora da base.\nFavor adicionar url deste à base.\n')
        return -1

    user_url = user['url'].iloc[0]

    response = get(f'{user_url}{archives}')
    datas = response.json()
    datas = datas['archives']


    # gets every game played
    player_games = []

    for data in datas:
        response = get(data)
        monthly_games = response.json()['games']
        player_games.extend(monthly_games)

    player_games = pd.DataFrame(player_games)
    player_games['player'] = u


    # processes the response
    player_games = player_games.apply(extract_black_white,axis=1)
    player_games.drop(columns=['white','black'],inplace=True)
    player_games['end_time'] = pd.to_datetime(player_games['end_time'], unit='s')


    # annotates the user info
    users.loc[users['username'] == u, 'gathered'] = True
    adds_user_info(player_games, u)

    return player_games

## Seção de testes

```
import asyncio

from chessdotcom.aio import get_player_profile, Client
#or
from chessdotcom import Client
Client.aio = True

usernames = ["fabianocaruana", "GMHikaruOnTwitch", "MagnusCarlsen", "GarryKasparov"]

cors = [get_player_profile(name) for name in usernames]

async def gather_cors(cors):
   responses = await asyncio.gather(*cors)
   return responses

responses = asyncio.run(gather_cors(cors))

# api_url = "https://api.chess.com/pub/player/"
```

In [ ]:
if test:
    users = [
        {
        'username':'Mr-Barros',
        'url':'https://api.chess.com/pub/player/mr-barros',
        'gathered': False,
        }
    ]

    users = pd.DataFrame(users)

### acessa os {jogador}/games/archives, pega todos os links

In [ ]:
if test:
    archives = "/games/archives"
    u = users['username'][0]
    user_url = users.loc[users['username'] == u, 'url'][0]

    response = get(f'{user_url}{archives}')
    datas = response.json()
    datas = datas['archives']

### acessa cada um dos links e baixa o pgn de resposta de todos, acumulando ele em um único pgn por usuario ao terminar muda o valor no dicionario de users para True

In [ ]:
if test:
    player_games = []

    for data in datas:
        response = get(data)
        monthly_games = response.json()['games']
        player_games.extend(monthly_games)

    player_games = pd.DataFrame(player_games)
    player_games['player'] = u

In [ ]:
if test:
    player_games = player_games.apply(extract_black_white,axis=1)
    player_games.drop(columns=['white','black'],inplace=True)
    player_games['end_time'] = pd.to_datetime(player_games['end_time'], unit='s')

### Adiciona informações do usuário trazido:

In [ ]:
if test:
    users.loc[users['username'] == u, 'gathered'] = True

    most_played_class = player_games['time_class'].mode()[0]
    latest_time = player_games.loc[player_games['end_time'] == player_games['end_time'].max(), 'end_time'].iloc[0]
    users.loc[users['username'] == u, 'most_played_class'] = most_played_class
    users.loc[users['username'] == u, 'latest_game'] = latest_time

    for time_class in player_games['time_class'].unique():
        games = player_games[player_games['time_class'] == time_class]
        amount_played = games.shape[0]
        latest_game = games.loc[games['end_time'] == games['end_time'].max()].iloc[0]
        latest_time = latest_game['end_time']
        p_rating = latest_game['player_rating']
        users.loc[users['username'] == u, f'{time_class}_rating'] = p_rating
        users.loc[users['username'] == u, f'{time_class}_played'] = amount_played
        users.loc[users['username'] == u, f'{time_class}_recent'] = latest_time



In [ ]:
if test:
    player_games.to_csv('../../dados/experimentos/chess_games_chesscom.csv', encoding='utf-8', index=False)
    users.to_csv('../../dados/experimentos/users_gathered.csv', encoding='utf-8', index=False)

# Seção para uso

**TROCAR A VARIÁVEL ```test``` PARA ```False``` APÓS RODAR TUDO PELA PRIMEIRA VEZ**

<span style="color:red;">**AVISO:**</span> Este código pode demorar muito para rodar para cada jogador, se algum jogador tiver muitos jogos, pois a API do chess.com é realmente lerda para responder. A ideia é no futuro paralelizar as requests para diminuir o tempo necessário para conseguir todos os jogos de um jogador.

In [ ]:
# number of players to newly gather games from
N = 3

users_yet_df = users[~users['gathered']]
users_yet = users_yet_df['username'].sample(n=N)

for user in users_yet:
    player_g = gather_player(user)
    if type(player_g) == int:
        continue
    df = pd.concat([df, player_g], axis=0)

In [ ]:
df.to_csv('../../dados/experimentos/chess_games_chesscom.csv', encoding='utf-8', index=False)
users.to_csv('../../dados/experimentos/users_gathered.csv', encoding='utf-8', index=False)